# Affiliation Rate Dashboard

**NOTE: The output of some cells is not displayed to protect sensitive information**

## Conexión y librerías

In [1]:
import sys
sys.path.append('../')
import conexion_database

No Laptop Tandamos
Spartan PC


In [2]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np

# Para guardar los archivos con la fecha de hoy
from datetime import datetime, date, timedelta

# Limpieza de query
from clean_query import clean_query

# La función de weeknum es equivalente a la de WEEKNUM en Excel, modo 1, en la que se basa Alejo
from calweek import weeknum

In [3]:
# Conexión con Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

scopes = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]
credentials = ServiceAccountCredentials.from_json_keyfile_name("../gsheets_key.json", scopes) #access the json key you downloaded earlier 
client = gspread.authorize(credentials) # authenticate the JSON key with gspread

## Load data

**Graph - Afiliation rate evolution**
* Affiliated. `app_user_onboarding_approved_date` y `terms_conditions_accepted_date` no sean nulos. TAUO
* Fecha de afiliación. El campo más grande entre `app_user_onboarding_approved_date` y `terms_conditions_accepted_date`. TAUO
* Cuántos días pasaron desde que se registraron (registered): `signup_cohort_date` TAU
* Cohorte (semana en que se registraron): cálculo manual basado en `signup_cohort_date` TAU

**Highlevel funnel snapshot**
- Downloads: Google Analytics; se refiere a la 1a vez que la gente abre la aplicación
- Signups: `tau.phone` no nulo,  `tmfu.flow_type`: “sms”, `tmfu.event_name`: “verification_completed”
- Approvals: `approved_cohort_date`
- Affiliated: ver arriba
- Conversions: `conversion_cohort_date` TAU

**Affiliation Rate evolution by Cohort:**
- App Downloads: Google Analytics
- Registered Email: `email` TAU
- Started Ph. Verif: `tmfu.flow_type`: “sms”, `event_name` = "verification_started" TMFU (por usuario único)
- Passed Ph. Verif: `tmfu.flow_type`: “sms”, `event_name` = "verification_completed" TMFU (por usuario único)
- Sign Ups: ver arriba

**Affiliation Rate evolution by Cohort: (2)**
- Approbals: `approved_cohort_date`
- Started onboarding: TMFU `flow_type` = "onboarding" y `event_name` = "verification_started" (por usuario único)
- Finished: TMFU `flow_type` = "onboarding" y `event_name` = "verification_completed" (por usuario único
- T&C and Contract: `terms_and_conditions_accepted_date` no nulo TAUO
- Passed: `onboarding_approved_date` no nulo TAUO
- Affiliated: ver arriba

### Data tau-tauo

In [4]:
# Redactar SQL query
query = """
SELECT tau.id, name, email, phone, signup_cohort_date, approved_cohort_date, conversion_cohort_date, app_user_onboarding_approved_date AS onboarding_approved_date, terms_conditions_accepted_date AS terms_accepted_date
FROM tdm_app_user tau 
	JOIN tdm_app_user_onboarding tauo ON tau.id = tauo.app_user_id
WHERE tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440, 2432, 2442, 2443, 2444, 2445, 2447, 2446, 2450, 2451, 2452,  2454, 2455, 2477, 3227, 4773, 4776, 4816, 7572, 12912, 13697, 19651)
;
"""
query = clean_query(query)
print(query)

SELECT tau.id, name, email, phone, signup_cohort_date, approved_cohort_date, conversion_cohort_date, app_user_onboarding_approved_date AS onboarding_approved_date, terms_conditions_accepted_date AS terms_accepted_date FROM tdm_app_user tau JOIN tdm_app_user_onboarding tauo ON tau.id = tauo.app_user_id WHERE tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440, 2432, 2442, 2443, 2444, 2445, 2447, 2446, 2450, 2451, 2452, 2454, 2455, 2477, 3227, 4773, 4776, 4816, 7572, 12912, 13697, 19651) ; 


In [ ]:
# Cargar query a un DataFrame
df_tau_tauo = pd.read_sql_query(query, conexion_database.engine)
print(df_tau_tauo.shape)
df_tau_tauo.head(4)

In [6]:
# Agregar columnas de status de afiliado y de fecha de afiliación
df_tau_tauo['fecha_afiliacion'] = df_tau_tauo[['onboarding_approved_date', 'terms_accepted_date']].max(axis=1, skipna=False)
df_tau_tauo['affiliated'] = np.where(df_tau_tauo.onboarding_approved_date.notnull() & df_tau_tauo.terms_accepted_date.notnull(), 'afiliado', 'no_afiliado')

# Agregar columna con el año solito
df_tau_tauo['signup_year'] = df_tau_tauo.signup_cohort_date.dt.year

# Agregar semana de cohorte (para ello uso la función weeknum() que imita WEEKNUM de Excel)
df_tau_tauo['signup_week'] = df_tau_tauo.signup_cohort_date.apply(weeknum)

### Data tau_mati

In [8]:
query = """
SELECT tau.id, email, name, phone, signup_cohort_date, flow_type, event_name, tmfu.insert_date
FROM tdm_app_user tau 
	JOIN tdm_mati_follow_up tmfu ON tau.id = tmfu.app_user_id
WHERE tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440, 2432, 2442, 2443, 2444, 2445, 2447, 2446, 2450, 2451, 2452,  2454, 2455, 2477, 3227, 4773, 4776, 4816, 7572, 12912, 13697, 19651)
;
"""
query = clean_query(query)
print(query)

SELECT tau.id, email, name, phone, signup_cohort_date, flow_type, event_name, tmfu.insert_date FROM tdm_app_user tau JOIN tdm_mati_follow_up tmfu ON tau.id = tmfu.app_user_id WHERE tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440, 2432, 2442, 2443, 2444, 2445, 2447, 2446, 2450, 2451, 2452, 2454, 2455, 2477, 3227, 4773, 4776, 4816, 7572, 12912, 13697, 19651) ; 


In [ ]:
# Cargar query a un DataFrame
df_tau_mati = pd.read_sql_query(query, conexion_database.engine)
print(df_tau_mati.shape)
df_tau_mati.head(4)

## 1. Gráfica - Afiliation rate evolution

### Preparar data

In [10]:
data_are = df_tau_tauo.copy()

In [11]:
# Cambiar columnas de fecha para que no incluyan la hora y los segundos
data_are['signup_cohort_date'] = data_are['signup_cohort_date'].dt.date
data_are['fecha_afiliacion'] = data_are['fecha_afiliacion'].dt.date

# Agregar columna de días en afiliarse
data_are['dias_en_afiliarse'] = (data_are.fecha_afiliacion - data_are.signup_cohort_date).dt.days

In [ ]:
data_are.head()

Vamos a sacar ahora la agrupación de días en afiliarse por cohorte. 

In [13]:
# Crear rango para mis bins
rango_bins_dias_en_afiliarse = np.arange(0, 36, 1)
rango_bins_dias_en_afiliarse = np.append(rango_bins_dias_en_afiliarse, 10000)
rango_bins_dias_en_afiliarse

array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
           9,    10,    11,    12,    13,    14,    15,    16,    17,
          18,    19,    20,    21,    22,    23,    24,    25,    26,
          27,    28,    29,    30,    31,    32,    33,    34,    35,
       10000])

In [14]:
# Crear labels para mis bins
labels_rango_bins_dias_en_afiliarse = [str(i) for i in np.arange(0, 35, 1)]
labels_rango_bins_dias_en_afiliarse.append('35+')
print(labels_rango_bins_dias_en_afiliarse)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35+']


In [15]:
data_cohortes = data_are.groupby(['signup_year', 'signup_week', 
                  pd.cut(data_are["dias_en_afiliarse"], 
                         bins=rango_bins_dias_en_afiliarse, 
                         right=False, 
                         labels=labels_rango_bins_dias_en_afiliarse)])['id'].count().reset_index()

data_cohortes

,signup_year,signup_week,dias_en_afiliarse,id
0,2021,1,0,0
1,2021,1,1,0
2,2021,1,2,0
3,2021,1,3,0
4,2021,1,4,0
...,...,...,...,...
2083,2022,53,31,0
2084,2022,53,32,0
2085,2022,53,33,0
2086,2022,53,34,0


Por alguna razón me agregó más filas de las que quería, así que eliminemos las filas que no nos sirven

In [17]:
# Sacar año y semana actuales
año_actual = datetime.today().year
semana_actual = weeknum(datetime.today())

# Eliminar signup_weeks agregadas automáticamente por pandas
data_cohortes = data_cohortes[~((data_cohortes.signup_year == 2021) & (data_cohortes.signup_week < 44)) 
                                & ~((data_cohortes.signup_year == año_actual) & (data_cohortes.signup_week > semana_actual))].reset_index(drop=True)
data_cohortes

,signup_year,signup_week,dias_en_afiliarse,id
0,2021,44,0,4
1,2021,44,1,0
2,2021,44,2,0
3,2021,44,3,0
4,2021,44,4,0
...,...,...,...,...
1039,2022,19,31,0
1040,2022,19,32,0
1041,2022,19,33,0
1042,2022,19,34,0


Eliminar los cohortes para los cuales tenemos menos de 5 personas que se afiliaron:

In [19]:
data_cohortes_sin_data = data_are.groupby(['signup_year', 'signup_week'], as_index=False)['dias_en_afiliarse'].count()
data_cohortes_sin_data.rename({'dias_en_afiliarse': 'afiliados'}, axis=1, inplace=True)

zip_sin_data = data_cohortes_sin_data.loc[data_cohortes_sin_data.afiliados <= 5, ['signup_year', 'signup_week']]
lista_cohortes_sin_data = list(zip(zip_sin_data.signup_year, zip_sin_data.signup_week))
print(lista_cohortes_sin_data)

[(2021, 48), (2021, 49), (2022, 19)]


In [20]:
for cohorte in lista_cohortes_sin_data:
    data_cohortes = data_cohortes.loc[~((data_cohortes.signup_year == cohorte[0]) & (data_cohortes.signup_week == cohorte[1]))]

data_cohortes.signup_week.unique()

array([44, 45, 46, 47, 50, 51, 52, 53,  1,  2,  3,  4,  5,  6,  7,  8,  9,
       10, 11, 12, 13, 14, 15, 16, 17, 18], dtype=int64)

Ahora hay que sacar los días en afiliarse acumulados

In [21]:
data_cohortes['dias_en_afiliarse_cumulative'] = data_cohortes.groupby(['signup_year', 'signup_week'])['id'].cumsum()

Agregar columna con el total de cada cohorte, para después sacar porcentajes:

In [ ]:
# Tabla con singups totales por cohorte
signups_total_por_cohorte = data_are[data_are.signup_cohort_date.notnull()].groupby(['signup_year', 'signup_week'], as_index=False)['id'].count()
signups_total_por_cohorte.rename({'id': 'total_signups_cohorte'}, axis=1, inplace=True)
signups_total_por_cohorte.head()

In [ ]:
# Unir con la tabla general de data_cohortes
data_cohortes_per = data_cohortes.merge(signups_total_por_cohorte[['signup_year', 'signup_week', 'total_signups_cohorte']], how='left', left_on=['signup_year','signup_week'], right_on=['signup_year','signup_week'])
data_cohortes_per.head()

In [ ]:
data_cohortes_per['dias_en_afiliarse_total_percentage'] = (data_cohortes_per['dias_en_afiliarse_cumulative'] / data_cohortes_per['total_signups_cohorte'] * 100).round(1)
data_cohortes_per

Data final:

In [27]:
data_cohortes_per.head()

,signup_year,signup_week,dias_en_afiliarse,id,dias_en_afiliarse_cumulative,total_signups_cohorte,dias_en_afiliarse_total_percentage
0,2021,44,0,4,4,302,1.3
1,2021,44,1,0,4,302,1.3
2,2021,44,2,0,4,302,1.3
3,2021,44,3,0,4,302,1.3
4,2021,44,4,0,4,302,1.3


### Transformar data para graficación

Tengo que transformar la data a una two-way table.

|   | 44  | 45  | 46  |
|---|-----|-----|-----|
| 0 | val | ... | ... |
| 1 | val | ... | ... |
| 2 | val | ... | ... |

In [ ]:
cohort_pivot = pd.pivot_table(data_cohortes_per, values='dias_en_afiliarse_total_percentage', index='dias_en_afiliarse', columns=['signup_year', 'signup_week'])
cohort_pivot.columns = cohort_pivot.columns.map(lambda x: '|'.join([str(i) for i in x]))
cohort_pivot.reset_index(inplace=True)
cohort_pivot

Hacer un trim final: conservar solo los días que realmente han transcurrido para los últimos cohortes. 

In [30]:
# Eliminar los días que todavía no suceden
for i in range(len(cohort_pivot) // 7): 
    i += 1
    cohort_pivot.iloc[7*i:, -i] = ""

In [31]:
cohort_pivot

,dias_en_afiliarse,2021|44,2021|45,2021|46,2021|47,2021|50,2021|51,2021|52,2021|53,2022|1,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18
0,0,1.3,1.7,1.3,0.8,2.7,1.6,1.1,1.1,0.0,1.7,2.0,1.7,0.3,0.6,0.5,0.4,0.6,1.0,0.9,0.6,0.4,0.4,0.7,0.6,0.6,0.6
1,1,1.3,2.7,2.1,2.1,4.2,2.5,2.6,3.0,0.0,3.3,3.7,2.9,1.9,2.0,1.1,1.1,2.4,2.4,2.0,1.4,1.8,1.7,3.0,1.2,1.6,3.1
2,2,1.3,3.3,2.4,2.1,5.1,2.9,3.5,3.3,2.2,3.8,4.1,3.3,2.6,2.6,1.8,2.5,3.3,2.6,2.9,2.2,2.3,2.1,4.0,2.1,1.8,3.9
3,3,1.3,4.0,3.0,2.9,6.0,3.3,4.0,3.5,2.2,4.2,4.3,3.5,2.9,2.8,2.8,3.0,4.3,3.1,3.3,2.7,3.2,2.7,4.2,2.6,2.0,4.1
4,4,1.3,4.9,3.3,2.9,6.4,4.1,4.1,3.8,2.2,4.4,4.6,3.5,3.4,3.2,3.2,3.5,4.5,3.6,3.6,3.1,3.5,3.0,4.4,3.1,2.2,4.1
5,5,1.7,5.1,3.8,2.9,6.6,4.3,4.3,3.9,2.2,4.6,4.6,3.7,3.8,3.4,3.6,3.5,4.7,3.9,3.7,3.7,3.7,3.4,4.8,3.2,2.2,4.1
6,6,1.7,5.1,3.9,3.4,6.6,4.6,4.5,3.9,3.3,4.7,4.7,3.9,4.0,3.5,3.7,3.6,4.7,4.3,3.7,3.9,4.0,3.6,4.8,3.5,2.4,4.1
7,7,2.0,5.2,3.9,3.4,7.5,4.8,4.6,3.9,3.3,4.8,4.7,3.9,4.2,3.5,3.9,3.7,4.8,4.3,3.8,4.1,4.0,4.0,5.1,3.6,2.4,
8,8,2.3,5.3,3.9,3.8,7.5,4.9,4.6,4.0,3.3,4.8,4.7,3.9,4.3,3.6,3.9,3.8,5.0,4.4,3.8,4.2,4.2,4.2,5.5,3.6,2.4,
9,9,2.3,5.3,4.1,4.2,7.7,5.1,4.6,4.1,3.3,4.8,4.7,4.0,4.4,3.6,3.9,4.2,5.0,4.5,3.8,4.6,4.3,4.2,5.8,3.6,2.4,


### Pasar a G Sheets

Incluir filtro por fecha

In [32]:
sheet = client.open("Business_Health_Dashboard")  #open sheet

#replace sheet_name with the name that corresponds to yours, e.g, it can be sheet1
sheet = sheet.worksheet("data_afiliation_rate")

In [33]:
sheet.update([cohort_pivot.columns.values.tolist()] + cohort_pivot.values.tolist())

{'spreadsheetId': '1ZKpgLgKOSt-8JuokkfHOcTBAG72ER73-RcE-g7sttJ4',
 'updatedRange': 'data_afiliation_rate!A1:AA37',
 'updatedRows': 37,
 'updatedColumns': 27,
 'updatedCells': 999}

## 2. Highlevel Funnel Snapshot

### A) Números históricos

In [34]:
numeros_funnel_df = pd.DataFrame({'funnel_phase': ['downloads', 'signups', 'approvals', 'affiliated', 'converted', 'downloads', 'emails', 'started_verif', 'passed_verif', 'signups', 'approvals', 'started_onboarding', 'finished_onboarding', 'signed_terms', 'passed_onboarding', 'affiliated'],
                                  'num_historico': '',
                                })
numeros_funnel_df.head()

,funnel_phase,num_historico
0,downloads,
1,signups,
2,approvals,
3,affiliated,
4,converted,


#### Fase de Download-Conversion

In [35]:
data_hfs = df_tau_tauo.copy()

In [36]:
# Obtener downloads
sheet = client.open("Business_Health_Dashboard")  #open sheet

#replace sheet_name with the name that corresponds to yours, e.g, it can be sheet1
sheet = sheet.worksheet("data_highlevel_funnnel_snapshot_pivot")

downloads_sheets = sheet.acell('G2').value

In [37]:
# Obtener cifras crudas
downloads = int(downloads_sheets)
signups =  df_tau_mati[(df_tau_mati.phone.notnull()) & (df_tau_mati.flow_type == 'sms') & (df_tau_mati.event_name == 'verification_completed')].drop_duplicates('id').id.count()
approvals = data_hfs.approved_cohort_date.count() 
affiliated = data_hfs.fecha_afiliacion.count()
conversions = data_hfs.conversion_cohort_date.count() 

#### Fase de Download-Signup

In [38]:
# Obtener cifras crudas
emails = data_hfs.signup_cohort_date.count()
started_verif = df_tau_mati[(df_tau_mati.flow_type == 'sms') & (df_tau_mati.event_name == 'verification_started')].drop_duplicates('id').id.count()
passed_verif = df_tau_mati[(df_tau_mati.flow_type == 'sms') & (df_tau_mati.event_name == 'verification_completed')].drop_duplicates('id').id.count()

In [ ]:
# Sacar porcentajes de cambio
downloads_to_email = emails / downloads
email_to_started_verif = started_verif / emails
started_verif_to_passed_verif = passed_verif / started_verif
passed_verif_to_signup  = signups / passed_verif

# Check que números vayan decreciendo
lista_download_signup = [downloads, emails, started_verif, passed_verif, signups]
for i in lista_download_signup:
    print("{:,}".format(i))

#### Fase de Approval-Affiliated

In [40]:
# Obtener cifras crudas
started_onboarding = df_tau_mati[(df_tau_mati.flow_type == 'onboarding') & (df_tau_mati.event_name == 'verification_started')].drop_duplicates('id').id.count()
finished_onboarding = df_tau_mati[(df_tau_mati.flow_type == 'onboarding') & (df_tau_mati.event_name == 'verification_completed')].drop_duplicates('id').id.count()
signed_terms = df_tau_tauo.terms_accepted_date.count()
passed_onboarding = df_tau_tauo.onboarding_approved_date.count()

In [ ]:
# Registrar números históricos en DataFrame
items_funnel = [downloads, signups, approvals, affiliated, conversions,
               downloads, emails, started_verif, passed_verif, signups,
               approvals, started_onboarding, finished_onboarding, signed_terms, passed_onboarding, affiliated]


numeros_funnel_df['num_historico'] = pd.Series(items_funnel)
numeros_funnel_df

#### Pasar a G Sheets

In [42]:
#Open sheet
sheet = client.open("Business_Health_Dashboard") 

#replace sheet_name with the name that corresponds to yours, e.g, it can be sheet1
sheet_averages = sheet.worksheet("data_highlevel_funnnel_snapshot_pivot")
sheet_averages.update([numeros_funnel_df.columns.values.tolist()] + numeros_funnel_df.values.tolist())

{'spreadsheetId': '1ZKpgLgKOSt-8JuokkfHOcTBAG72ER73-RcE-g7sttJ4',
 'updatedRange': 'data_highlevel_funnnel_snapshot_pivot!A1:B17',
 'updatedRows': 17,
 'updatedColumns': 2,
 'updatedCells': 34}

### C) Números por día

**IMPORTANTE: Las cifras deben estar en relación con los cohortes de registro. Así, por ejemplo, el número de signups o aprovals  en debe ser para los usuarios que se registraron en las fechas seleccionadas.** 

In [70]:
numeros_dia = pd.DataFrame({'dia': pd.date_range(start='2021-10-12', end=date.today().strftime('%Y-%m-%d')),
                           })
numeros_dia.head()

,dia
0,2021-10-12
1,2021-10-13
2,2021-10-14
3,2021-10-15
4,2021-10-16


#### Fase de Download-Conversion

In [71]:
data_por_dia = df_tau_tauo.copy()

In [ ]:
data_por_dia.head()

In [73]:
downloads = 0
signups = df_tau_mati[(df_tau_mati.phone.notnull()) & (df_tau_mati.flow_type == 'sms') & (df_tau_mati.event_name == 'verification_completed')].drop_duplicates('id').resample('D', on='signup_cohort_date')['id'].count()
approvals = data_por_dia[data_por_dia.approved_cohort_date.notnull()].resample('D', on='signup_cohort_date')['id'].count()
affiliated = data_por_dia[data_por_dia.fecha_afiliacion.notnull()].resample('D', on='signup_cohort_date')['id'].count()
conversions = data_por_dia[data_por_dia.conversion_cohort_date.notnull()].resample('D', on='signup_cohort_date')['id'].count()

#### Fase de Download-Signup

In [74]:
emails = data_por_dia.resample('D', on='signup_cohort_date')['id'].count()
started_verif = df_tau_mati[(df_tau_mati.flow_type == 'sms') & (df_tau_mati.event_name == 'verification_started')].drop_duplicates('id').resample('D', on='signup_cohort_date')['id'].count()
passed_verif = df_tau_mati[(df_tau_mati.flow_type == 'sms') & (df_tau_mati.event_name == 'verification_completed')].drop_duplicates('id').resample('D', on='signup_cohort_date')['id'].count()

#### Fase de Approval-Affiliated

In [75]:
started_onboarding = df_tau_mati[(df_tau_mati.flow_type == 'onboarding') & (df_tau_mati.event_name == 'verification_started')].drop_duplicates('id').resample('D', on='signup_cohort_date')['id'].count()
finished_onboarding = df_tau_mati[(df_tau_mati.flow_type == 'onboarding') & (df_tau_mati.event_name == 'verification_completed')].drop_duplicates('id').resample('D', on='signup_cohort_date')['id'].count()
signed_terms = df_tau_tauo[df_tau_tauo.terms_accepted_date.notnull()].resample('D', on='signup_cohort_date')['id'].count()
passed_onboarding = df_tau_tauo[df_tau_tauo.onboarding_approved_date.notnull()].resample('D', on='signup_cohort_date')['id'].count()

#### Llenar DataFrame

In [76]:
# Fase de Download-Conversion
numeros_dia['downloads'] = downloads
numeros_dia = numeros_dia.merge(signups, how='left', left_on='dia', right_index=True).rename(columns={'id': 'signups'})
numeros_dia = numeros_dia.merge(approvals, how='left', left_on='dia', right_index=True).rename(columns={'id': 'approvals'})
numeros_dia = numeros_dia.merge(affiliated, how='left', left_on='dia', right_index=True).rename(columns={'id': 'affiliated'})
numeros_dia = numeros_dia.merge(conversions, how='left', left_on='dia', right_index=True).rename(columns={'id': 'conversions'})

# Fase de Download-Signup
numeros_dia = numeros_dia.merge(emails, how='left', left_on='dia', right_index=True).rename(columns={'id': 'emails'})
numeros_dia = numeros_dia.merge(started_verif, how='left', left_on='dia', right_index=True).rename(columns={'id': 'started_verif'})
numeros_dia = numeros_dia.merge(passed_verif, how='left', left_on='dia', right_index=True).rename(columns={'id': 'passed_verif'})

# Fase de Approval-Affiliated
numeros_dia = numeros_dia.merge(started_onboarding, how='left', left_on='dia', right_index=True).rename(columns={'id': 'started_onboarding'})
numeros_dia = numeros_dia.merge(finished_onboarding, how='left', left_on='dia', right_index=True).rename(columns={'id': 'finished_onboarding'})
numeros_dia = numeros_dia.merge(signed_terms, how='left', left_on='dia', right_index=True).rename(columns={'id': 'signed_terms'})
numeros_dia = numeros_dia.merge(passed_onboarding, how='left', left_on='dia', right_index=True).rename(columns={'id': 'passed_onboarding'})

# Llenar los nulos
numeros_dia.fillna(0, inplace=True)

# Transformar columna datetime a string
numeros_dia['dia'] = numeros_dia['dia'].dt.strftime("%Y-%m-%d")

In [ ]:
numeros_dia

#### Pasar a G Sheets

In [78]:
#Open sheet
sheet = client.open("Business_Health_Dashboard") 

#replace sheet_name with the name that corresponds to yours, e.g, it can be sheet1
sheet_averages = sheet.worksheet("data_highlevel_funnel_snapshot")
sheet_averages.update([numeros_dia.columns.values.tolist()] + numeros_dia.values.tolist())

{'spreadsheetId': '1ZKpgLgKOSt-8JuokkfHOcTBAG72ER73-RcE-g7sttJ4',
 'updatedRange': 'data_highlevel_funnel_snapshot!A1:M206',
 'updatedRows': 206,
 'updatedColumns': 13,
 'updatedCells': 2678}